In [28]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv('finviz.csv')

In [30]:
df.head()

,No.,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Forward P/E,PEG,...,Gap,Analyst Recom,Average Volume,Relative Volume,Price,Change,Volume,Earnings Date,Target Price,IPO Date
0,1,A,"Agilent Technologies, Inc.",Healthcare,Medical Laboratories & Research,USA,26597.29,25.78,22.75,2.81,...,-1.99%,2.2,1625.70,1.01,86.56,-1.93%,1646009,2/24/2020 4:30:00 PM,88.81,11/18/1999
1,2,AA,Alcoa Corporation,Basic Materials,Aluminum,USA,3131.92,NaN,14.22,NaN,...,-3.16%,2.7,4424.20,1.49,14.87,-4.13%,6581047,1/15/2020 4:30:00 PM,23.77,1/2/1962
2,3,AAAU,Perth Mint Physical Gold ETF,Financial,Exchange Traded Fund,USA,NaN,NaN,NaN,NaN,...,0.80%,NaN,68.63,0.83,15.79,0.73%,56818,NaN,NaN,8/15/2018
3,4,AACG,ATA Creativity Global,Services,Education & Training Services,China,36.65,NaN,NaN,NaN,...,2.21%,NaN,18.05,0.51,1.31,-3.68%,9179,NaN,7.00,10/17/2019
4,5,AADR,AdvisorShares Dorsey Wright ADR ETF,Financial,Exchange Traded Fund,USA,NaN,NaN,NaN,NaN,...,-2.00%,NaN,13.35,0.69,54.37,-1.91%,9273,NaN,NaN,7/21/2010


In [31]:
df.shape

(7823, 71)

In [32]:
df.columns

Index(['No.', 'Ticker', 'Company', 'Sector', 'Industry', 'Country',
       'Market Cap', 'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'P/Cash',
       'P/Free Cash Flow', 'Dividend Yield', 'Payout Ratio', 'EPS (ttm)',
       'EPS growth this year', 'EPS growth next year',
       'EPS growth past 5 years', 'EPS growth next 5 years',
       'Sales growth past 5 years', 'EPS growth quarter over quarter',
       'Sales growth quarter over quarter', 'Shares Outstanding',
       'Shares Float', 'Insider Ownership', 'Insider Transactions',
       'Institutional Ownership', 'Institutional Transactions', 'Float Short',
       'Short Ratio', 'Return on Assets', 'Return on Equity',
       'Return on Investment', 'Current Ratio', 'Quick Ratio',
       'LT Debt/Equity', 'Total Debt/Equity', 'Gross Margin',
       'Operating Margin', 'Profit Margin', 'Performance (Week)',
       'Performance (Month)', 'Performance (Quarter)',
       'Performance (Half Year)', 'Performance (Year)', 'Performance (YTD)',

In [33]:
df['AnalystCoverage'] = df['Analyst Recom'].apply(lambda x: 1 if x > 0 else 0)

In [34]:
df['AnalystCoverage']

0       1
1       1
2       0
3       0
4       0
       ..
7818    1
7819    1
7820    1
7821    1
7822    1
Name: AnalystCoverage, Length: 7823, dtype: int64

In [35]:
df['LogMarketCap'] = df['Market Cap'].apply(lambda x: np.log(1+x))

In [39]:
df['InstOwner'] = df['Institutional Ownership'].apply(lambda x: float(str(x).strip('%'))/100)

In [40]:
df['LogVolume'] = df['Volume'].apply(lambda x: np.log(1+x))

In [41]:
import statsmodels.api as sm

In [42]:
from patsy import dmatrices

In [43]:
df = df[['AnalystCoverage', 'LogMarketCap', 'InstOwner', 'LogVolume']].dropna()

In [44]:
df

,AnalystCoverage,LogMarketCap,InstOwner,LogVolume
0,1,10.188602,0.884,14.313865
1,1,8.049721,0.898,15.699705
3,0,3.628333,0.089,9.124782
5,1,9.353523,0.815,16.883809
6,0,3.090133,0.203,7.314553
...,...,...,...,...
7818,1,7.428167,0.656,13.651182
7819,1,3.875152,0.740,10.568878
7820,1,7.457246,0.646,13.153249
7821,1,4.827674,0.476,12.729721


In [55]:
y, X = dmatrices('AnalystCoverage ~ LogMarketCap + InstOwner + LogVolume', data=df, return_type='dataframe')

In [56]:
X

,Intercept,LogMarketCap,InstOwner,LogVolume
0,1.0,10.188602,0.884,14.313865
1,1.0,8.049721,0.898,15.699705
3,1.0,3.628333,0.089,9.124782
5,1.0,9.353523,0.815,16.883809
6,1.0,3.090133,0.203,7.314553
...,...,...,...,...
7818,1.0,7.428167,0.656,13.651182
7819,1.0,3.875152,0.740,10.568878
7820,1.0,7.457246,0.646,13.153249
7821,1.0,4.827674,0.476,12.729721


In [57]:
model = sm.OLS(y, X)

In [58]:
results = model.fit()

In [59]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        AnalystCoverage   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     767.3
Date:                Wed, 05 Feb 2020   Prob (F-statistic):               0.00
Time:                        05:32:35   Log-Likelihood:                -2030.0
No. Observations:                4731   AIC:                             4068.
Df Residuals:                    4727   BIC:                             4094.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.3537      0.027    -12.902      0.000      -0.407      -0.300
LogMarketCap     0.0190      0.003      6.271      0.000       0.013       0.025
InstOwner        0.4147      0.019     21.368      0.000       0.377       0.453
LogVolume        0.0599      0.003     22.386      0.000       0.055       0.065
==============================================================================
Omnibus:                      186.299   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              158.248
Skew:                          -0.378   Prob(JB):                     4.33e-35
Kurtosis:                       2.520   Cond. No.                         71.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
df.head()

,AnalystCoverage,LogMarketCap
0,1,10.188602
1,1,8.049721
3,0,3.628333
5,1,9.353523
6,0,3.090133
